In [1]:
import pandas as pd
import numpy as np
import catllm as cat
import json
import os
import regex
import glob
import time
from dotenv import load_dotenv, find_dotenv
from pathlib import Path

In [2]:
os.chdir('/Users/chrissoria/Documents/Research/Categorization_AI_experiments')
current_directory = os.getcwd()

dotenv_path = Path('/Users/chrissoria/Documents/Research/Categorization_AI_experiments/.env')
pictures_path = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_Image_Files"
latest_cog_path = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/Data/CADAS data upload/Puerto_Rico/Sync_files/2025_05_30/EXCEL/cognitive.xlsx"
save_path = "/Users/chrissoria/Google Drive/other computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/Figures/AI_Scores"

load_dotenv(dotenv_path=dotenv_path)

open_ai_key = os.environ.get("OPENAI_API_KEY")

read in Puerto Rico data

In [3]:
pr_cog = pd.read_excel(latest_cog_path)
print(f"The number of rows in the Puerto Rico cognitive data before dropping non-resumen matches is: {len(pr_cog)}")
pr_cog = pr_cog[pr_cog['resumen'] == "Found in Resumen"].reset_index(drop=True)
print(f"The number of rows in the Puerto Rico cognitive data after dropping non-resumen matches is: {len(pr_cog)}")
pr_cog.head()

The number of rows in the Puerto Rico cognitive data before dropping non-resumen matches is: 372
The number of rows in the Puerto Rico cognitive data after dropping non-resumen matches is: 203


,pid_parent,pid,resumen,pid_nonmatch,globalrecordid,hhid,fkey,c_parent_houseid,c_parent_clustid,c_parent_particid,...,date_greater_102423,c_countmissing,c_last,c_ThreeWordDelay,c_TenWordDelay,c_TenWordRecognition,c_Story1Delay,c_Story2Delay,c_FigureDelay,c_TotalTime
0,00100101,100101,Found in Resumen,NaN,d6695ddc-846c-4ceb-a35e-c83c42cd76f1,1001,b3e344d0-5b45-47b8-9308-23e2b4f33f45,1.0,1.0,1.0,...,1,38,pent_pic_cropped,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00.00.0.,100101,Found in Resumen,NaN,26cc442c-7bf0-4790-80a1-ba315ac8c5c2,1001,a03580ab-4742-499a-98b2-fb517db7c217,NaN,NaN,NaN,...,1,9,pent_pic_cropped,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00100101,100101,Found in Resumen,NaN,43a6c5fe-d5c9-4c2a-a0a1-974bb77821f9,1001,b07cb4f9-0d36-409a-b7d6-00a472ab1b7c,1.0,1.0,1.0,...,1,170,pent_pic_cropped,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00100101,100101,Found in Resumen,NaN,18defbe6-f574-45e5-aab4-a2810aaeee15,1001,eb4f5349-93ae-40f2-96f3-e1f75d465647,1.0,1.0,1.0,...,1,5,pent_pic_cropped,2.0,12.0,17.0,10.0,10.0,6.0,44.0
4,00100101,100101,Found in Resumen,NaN,6fd62269-18f2-4ccc-a03a-54515f710efd,1001,608cca02-3db1-479c-8ec9-255d78934600,1.0,1.0,1.0,...,1,92,pent_pic_cropped,1.0,6.0,9.0,5.0,5.0,NaN,NaN


defining the function that will process the pictures columns

In [4]:
def process_pic_column(df, pic_col, pictures_path):
    pic_col = pic_col.lower()
    processed_pics = df[pic_col].apply(lambda x: str(x) if str(x).endswith('.jpg') else str(x) + '.jpg' if not pd.isna(x) else '')
    return pictures_path + '/' + processed_pics

#defining the pictures columns
pic_columns = ['c_72_1_pic', 'c_72_2_pic', 'c_72_3_pic', 'c_72_4_pic', 'c_79_1_pic', 'c_79_2_pic', 'c_79_3_pic', 'c_79_4_pic']

# process each picture column
for pic_col in pic_columns:
    pr_cog[f'{pic_col}_path'] = process_pic_column(pr_cog, pic_col, pictures_path)

pr_cog['c_72_1_pic_path'].head()

0    /Users/chrissoria/Google Drive/other computers...
1    /Users/chrissoria/Google Drive/other computers...
2    /Users/chrissoria/Google Drive/other computers...
3    /Users/chrissoria/Google Drive/other computers...
4    /Users/chrissoria/Google Drive/other computers...
Name: c_72_1_pic_path, dtype: object

In [ ]:
pr_c_79_1 = cat.cerad_score(
    shape="circle",
    image_input=pr_cog['c_79_1_pic_path'].tolist(),
    api_key=open_ai_key,
    safety=True,
    filename=f"{save_path}/c_79_1_machine_score_full.csv",
)

Provided a list of 203 images.


Categorising images: 100%|██████████| 203/203 [05:37<00:00,  1.67s/it]

An error occurred: <html>
<head><title>400 Bad Request</title></head>
<body>
<center><h1>400 Bad Request</h1></center>
<hr><center>nginx</center>
</body>
</html>


In [ ]:
pr_c_79_2 = cat.cerad_score(
    shape="diamond",
    image_input=pr_cog['c_79_2_pic_path'].tolist(),
    api_key=open_ai_key,
    safety=True,
    filename=f"{save_path}/c_79_2_machine_score_full.csv",
)

Provided a list of 203 images.


Categorising images: 100%|██████████| 203/203 [03:50<00:00,  1.14s/it]


In [ ]:
pr_c_79_3 = cat.cerad_score(
    shape="rectangles",
    image_input=pr_cog['c_79_3_pic_path'].tolist(),
    api_key=open_ai_key,
    safety=True,
    filename=f"{save_path}/c_79_3_machine_score_full.csv",
)

Provided a list of 203 images.


Categorising images: 100%|██████████| 203/203 [04:48<00:00,  1.42s/it]


In [7]:
pr_c_79_4 = cat.cerad_score(
    shape="cube",
    image_input=pr_cog['c_79_4_pic_path'].tolist(),
    api_key=open_ai_key,
    safety=True,
    user_model="gpt-4o",
    filename=f"{save_path}/c_79_4_machine_score_full.csv",
)

Provided a list of 203 images.


Categorising images: 100%|██████████| 203/203 [08:27<00:00,  2.50s/it]


In [6]:
pr_c_72_1 = cat.cerad_drawn_score(
    shape="circle",
    image_input=pr_cog['c_72_1_pic_path'].tolist(),
    api_key=open_ai_key,
    safety=True,
    reference_in_image=True,
    user_model="gpt-4o",
    filename=f"{save_path}/c_72_1_machine_score_full.csv",
)

Provided a list of 203 images.


Scoring images: 100%|██████████| 203/203 [13:09<00:00,  3.89s/it]


In [8]:
pr_c_72_2 = cat.cerad_drawn_score(
    shape="diamond",
    image_input=pr_cog['c_72_2_pic_path'].tolist(),
    api_key=open_ai_key,
    safety=True,
    reference_in_image=True,
    user_model="gpt-4o",
    filename=f"{save_path}/c_72_2_machine_score_full.csv",
)

Provided a list of 203 images.


Scoring images: 100%|██████████| 203/203 [13:42<00:00,  4.05s/it]
